In [6]:
%load_ext autoreload
%autoreload 
import pandas as pd
import numpy as np 
import GPy as GPy
from GPy_preferenceKernels import PreferenceKern
from sklearn.model_selection import train_test_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preference Kernel


In [44]:
#label preference dataset
df = pd.read_csv("datasets/DDModeChoice.txt", sep="\t", index_col=0)
df.drop_duplicates(inplace=True)#.reset_index(inplace=True)
df = df[(df.Season==0)&(df.CarAvail==0)&(df.Choice!=4) & (df.Grade>11)]#subselect data for homogeneity
df.loc[:,'Choice']=df.loc[:,'Choice']-1
#df.Choice:0:walk, 1:bike, 2:public transport

CovCol=['Distance', 'School_location', 'Grade', 'Age', 'Gender',
         'CB_location','Leistung']
Labels=[0,1,2]#possible labels
latent_dim= len(Labels)

df_train,df_test = train_test_split(df, test_size=0.3)

In [45]:
# train 
X_tr = df_train[CovCol].values
n = X_tr.shape[0]

pairs = []
for ii in range(df_train.shape[0]):
    choice = df_train.iloc[ii,0]
    diff = np.setdiff1d(Labels,choice)
    for d in diff:
        pairs.append([ii+n*choice,ii+n*d])
pairs_tr = np.vstack(pairs)

from sklearn.preprocessing import StandardScaler
scalerx = StandardScaler().fit(X_tr)
X_tr_n = scalerx.transform(X_tr)
Xaugm_tr = np.tile(X_tr_n,(latent_dim,1))
Xaugm_tr.shape

(723, 7)

In [46]:
X_twoargs=[]
Y=[]
for i in range(pairs_tr.shape[0]):
    X_twoargs.append(np.hstack([Xaugm_tr[pairs_tr[i][0]],Xaugm_tr[pairs_tr[i][1]]]))
    Y.append(1.0)
X_twoargs = np.vstack(X_twoargs)
Y = np.vstack(Y)


In [57]:
pairs_tr[i][1]

481

In [54]:
kernel.K(X_twoargs,X_twoargs)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [48]:
kernel = PreferenceKern(X_twoargs.shape[1], name="PreferenceKern", 
                        lengthscale=np.ones(int(X_twoargs.shape[1]/2)), variance=1.0, ARD=True)
inference = GPy.inference.latent_function_inference.Laplace()
likelihood = GPy.likelihoods.Bernoulli()
# Model definition
m = GPy.core.SparseGP(
    X_twoargs,Y, X_twoargs[0:5,:], kernel=kernel, likelihood=likelihood, inference_method=inference)

TypeError: inference() got multiple values for argument 'mean_function'

In [49]:
# Model definition
m = GPy.models.SparseGPClassification(X_twoargs,Y,  kernel=kernel)

LinAlgError: not pd: non-positive diagonal elements

In [ ]:
def run_model(X, Y, Xtest, typekernel="PreferenceKern",  type_inference="laplace", nrestarts=1, ARD=True):
    if typekernel=="PairGP":
        kernel=GPy.kern.RBF(X.shape[1],lengthscale=1.0*np.ones(X.shape[1]), variance=1.0, ARD=ARD)
    else: #*np.ones(int(X.shape[1]/2))
        kernel = PreferenceKern(X.shape[1], name=typekernel, lengthscale=np.ones(int(X.shape[1]/2)), variance=1.0, ARD=ARD) #*np.ones(int(X.shape[1]/2))
    if type_inference=="laplace":
        inference = GPy.inference.latent_function_inference.Laplace()
    elif type_inference=="EP":
        inference = GPy.inference.latent_function_inference.EP(ep_mode="nested")
    likelihood = GPy.likelihoods.Bernoulli()
    # Model definition
    m = GPy.core.GP(
        X,Y, kernel=kernel, likelihood=likelihood, inference_method=inference)
    #print(m)
    #m.kern.variance.constrain_fixed()
    if  typekernel=="GeneralPreferenceKern":
        m.optimize_restarts(nrestarts, optimizer="lbfgsb")
    else:
        m.optimize_restarts(nrestarts)
    #print(m)
    Ypred = (m.predict(Xtest)[0]>0.5)+0.0
    #print(m.kern.lengthscale)
    return Ypred, m